In [1]:
import torch
import torch.nn as nn
import numpy as np

# Master branch

In [2]:
class MasterNet(nn.Module):
    def __init__(self):
        super(MasterNet, self).__init__()
        
    def forward(self,x):
        return x

# Acoustic Branch

In [3]:
class AcousticNet(nn.Module):
    def __init__(self, num_conv_layers = 1):
        super(AcousticNet, self).__init__()
        self.conv = nn.Conv1d(in_channels=1, out_channels= 2, kernel_size=1)
        self.maxpool = nn.MaxPool1d(kernel_size = 2)
        self.relu = nn.ReLU()
        self.gru = nn.GRU(input_size=20,hidden_size=32,num_layers=2)
        self.mean_pool = nn.AvgPool1d(kernel_size=2)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x, _ = self.gru(x)
        x = self.mean_pool(x)
        return x

In [4]:
# Test dummy input
net = AcousticNet()
test_vec = torch.randn(10, 1, 40)
output = net(test_vec)
print(f'Shape of output: {output.shape}')
# assert output.shape[-1] == 16

Shape of output: torch.Size([10, 2, 16])


# Lexical Branch

In [5]:
# implement GRU (or transformer)
class LexicalNet(nn.Module):
    def __init__(self, num_gru_layers = 2):
        super(LexicalNet, self).__init__()
        # implement GRU (or transformer)
        self.gru = nn.GRU(input_size=300,hidden_size=32,num_layers=num_gru_layers)
        self.mean_pool = nn.AvgPool1d(kernel_size=2)
        
    def forward(self, x):
        x, _ = self.gru(x)
        x = self.mean_pool(x)
        return x

In [6]:
# Test dummy input
net = LexicalNet(num_gru_layers = 3)
test_vec = torch.randn(10, 1, 300)
output = net(test_vec)
assert output.shape[-1] == 16